In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
from scipy.spatial import ConvexHull

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/lexihenny/'
dir3='/Volumes/My Passport/'
dir2='/Volumes/Extreme Pro/'

In [ ]:
ds=xr.open_dataset(dir2+'merra2_gridareas.nc')
areas_merra2=ds['cell_area']/1e6

areas_list_era5_variable=[]
grids_list_era5_variable=[]
areas_list_era5_fixed=[]
grids_list_era5_fixed=[]
areas_list_era5_fixed_poleward=[]
grids_list_era5_fixed_poleward=[]
yrs=np.arange(1940,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)
        
    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')
        
    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values
        
    land_mask=dk['land']
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_merra2.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_era5_variable.append(sum(areas_year))
    grids_list_era5_variable.append(grids)
    
    ###
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_merra2.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_era5_fixed.append(sum(areas_year))
    grids_list_era5_fixed.append(grids)
    
    ###
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_merra2.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_era5_fixed_poleward.append(sum(areas_year))
    grids_list_era5_fixed_poleward.append(grids)

In [ ]:
ds=xr.open_dataset(dir2+'merra2_gridareas.nc')
areas_merra2=ds['cell_area']/1e6

areas_list_merra2_variable=[]
grids_list_merra2_variable=[]
areas_list_merra2_fixed=[]
grids_list_merra2_fixed=[]
areas_list_merra2_fixed_poleward=[]
grids_list_merra2_fixed_poleward=[]
yrs=np.arange(1980,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)
        
    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')
        
    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values
        
    land_mask=dk['land']
    
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_merra2.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_merra2_variable.append(sum(areas_year))
    grids_list_merra2_variable.append(grids)
    
    ###
    
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled'][0::2,:,:]
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_merra2.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_merra2_fixed.append(sum(areas_year))
    grids_list_merra2_fixed.append(grids)
    
    ###
    
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_merra2.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_merra2_fixed_poleward.append(sum(areas_year))
    grids_list_merra2_fixed_poleward.append(grids)

In [ ]:
fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)
ax.plot(np.arange(1940,2024,1),areas_list_era5_fixed)
plt.show()

In [ ]:
ds=xr.open_dataset(dir_data+'jra55_gridareas.nc')
areas_jra55=ds['cell_area']/1e6

areas_list_jra55_variable=[]
grids_list_jra55_variable=[]
areas_list_jra55_fixed=[]
grids_list_jra55_fixed=[]
areas_list_jra55_fixed_poleward=[]
grids_list_jra55_fixed_poleward=[]
yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)
        
    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')
        
    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values
        
    land_mask=dk['land']
    
    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    binary['lon']=areas_jra55.lon.values
    binary['lat']=areas_jra55.lat.values
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_jra55.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_jra55_variable.append(sum(areas_year))
    grids_list_jra55_variable.append(grids)
    
    ###
    
    ds=xr.open_dataset(dir_data+'jra55.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    binary['lon']=areas_jra55.lon.values
    binary['lat']=areas_jra55.lat.values
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_jra55.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_jra55_fixed.append(sum(areas_year))
    grids_list_jra55_fixed.append(grids)
    
    ###
    
    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    binary['lon']=areas_jra55.lon.values
    binary['lat']=areas_jra55.lat.values
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_jra55.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_jra55_fixed_poleward.append(sum(areas_year))
    grids_list_jra55_fixed_poleward.append(grids)

In [ ]:
fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)
ax.plot(yrs,areas_list_jra55_variable)
plt.show()

In [ ]:
ds=xr.open_dataset(dir_data+'merra2_gridareas.nc')
areas_merra2=ds['cell_area']/1e6

ds=xr.open_dataset('/Users/ahenny/Downloads/jra55_gridareas_125.nc')
areas_jra55=ds['cell_area']/1e6

areas_list_era5tp=[]
grids_list_era5tp=[]
areas_list_era5reid=[]
grids_list_era5reid=[]
areas_list_era5mk=[]
grids_list_era5mk=[]
areas_list_merra2lo=[]
grids_list_merra2lo=[]
areas_list_merra2cndl=[]
grids_list_merra2cndl=[]
areas_list_merra2ac=[]
grids_list_merra2ac=[]
areas_list_jra55gw=[]
grids_list_jra55gw=[]

yrs=np.arange(1980,2020,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)
        
    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')
        
    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values
        
    land_mask=dk['land']
    
    ###
    
    ds=xr.open_dataset(dir2+'era5.ar.tempestlr'+str(year)+'.nc')
    ar=ds['ar']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    binary=ar.where(land_mask_interp>=0.5)
    binary=binary.where(binary==1)
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_merra2.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_era5tp.append(sum(areas_year))
    grids_list_era5tp.append(grids)
    
    ###
    
    ds=xr.open_dataset(dir2+'era5.ar.reid500.'+str(year)+'.nc')
    ar=ds['ar']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    binary=ar.where(land_mask_interp>=0.5)
    binary=binary.where(binary==1)
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_merra2.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_era5reid.append(sum(areas_year))
    grids_list_era5reid.append(grids)
    
    ###
    
    ds=xr.open_dataset(dir2+'era5.ar.mundhenkv3.'+str(year)+'.nc')
    ar=ds['ar']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    binary=ar.where(land_mask_interp>=0.5)
    binary=binary.where(binary==1)
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_merra2.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_era5mk.append(sum(areas_year))
    grids_list_era5mk.append(grids)
    
    ###
    
    ds=xr.open_dataset(dir2+'merra2.ar.lorav2.'+str(year)+'.nc')
    ar=ds['ar']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    binary=ar.where(land_mask_interp>=0.5)
    binary=binary.where(binary==1)
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_merra2.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_merra2lo.append(sum(areas_year))
    grids_list_merra2lo.append(grids)
    
    ###
    
    ds=xr.open_dataset(dir2+'merra2.ar.cndl.'+str(year)+'.nc')
    ar=ds['ar']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    binary=ar.where(land_mask_interp>=0.5)
    binary=binary.where(binary==1)
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_merra2.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_merra2cndl.append(sum(areas_year))
    grids_list_merra2cndl.append(grids)
    
    ###
    
    ds=xr.open_dataset(dir2+'merra2.ar.acv2.'+str(year)+'.nc')
    ar=ds['ar']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    binary=ar.where(land_mask_interp>=0.5)
    binary=binary.where(binary==1)
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_merra2.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_merra2ac.append(sum(areas_year))
    grids_list_merra2ac.append(grids)
    
    ###
    
    ds=xr.open_dataset(dir2+'jra55.ar.gwv2.'+str(year)+'.nc')
    ar=ds['ar']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    binary=ar.where(land_mask_interp>=0.5)
    binary=binary.where(binary==1)
    binary['lon']=areas_jra55.lon.values
    binary['lat']=areas_jra55.lat.values
    
    areas_year=[]
    grids=binary.sum().values.tolist()
    for j in range(binary.time.size):
        area_sel=areas_jra55.where(binary[j,:,:]==1)
        area_sum=area_sel.sum().values.tolist()
        areas_year.append(area_sum)
    areas_list_jra55gw.append(sum(areas_year))
    grids_list_jra55gw.append(grids)

In [ ]:
dk=xr.Dataset()
dk['areas_list_era5_variable']=(('time'),areas_list_era5_variable)
dk['areas_list_era5_fixed']=(('time'),areas_list_era5_fixed)
dk['areas_list_era5_fixed_poleward']=(('time'),areas_list_era5_fixed_poleward)
dk['grids_list_era5_variable']=(('time'),grids_list_era5_variable)
dk['grids_list_era5_fixed']=(('time'),grids_list_era5_fixed)
dk['grids_list_era5_fixed_poleward']=(('time'),grids_list_era5_fixed_poleward)

dk['areas_list_merra2_variable']=(('time1'),areas_list_merra2_variable)
dk['areas_list_merra2_fixed']=(('time1'),areas_list_merra2_fixed)
dk['areas_list_merra2_fixed_poleward']=(('time1'),areas_list_merra2_fixed_poleward)
dk['grids_list_merra2_variable']=(('time1'),grids_list_merra2_variable)
dk['grids_list_merra2_fixed']=(('time1'),grids_list_merra2_fixed)
dk['grids_list_merra2_fixed_poleward']=(('time1'),grids_list_merra2_fixed_poleward)

dk['areas_list_jra55_variable']=(('time2'),areas_list_jra55_variable)
dk['areas_list_jra55_fixed']=(('time2'),areas_list_jra55_fixed)
dk['areas_list_jra55_fixed_poleward']=(('time2'),areas_list_jra55_fixed_poleward)
dk['grids_list_jra55_variable']=(('time2'),grids_list_jra55_variable)
dk['grids_list_jra55_fixed']=(('time2'),grids_list_jra55_fixed)
dk['grids_list_jra55_fixed_poleward']=(('time2'),grids_list_jra55_fixed_poleward)

dk['areas_list_era5tp']=(('time3'),areas_list_era5tp)
dk['grids_list_era5tp']=(('time3'),grids_list_era5tp)
dk['areas_list_era5reid']=(('time3'),areas_list_era5reid)
dk['grids_list_era5reid']=(('time3'),grids_list_era5reid)
dk['areas_list_era5mk']=(('time3'),areas_list_era5mk)
dk['grids_list_era5mk']=(('time3'),grids_list_era5mk)

dk['areas_list_merra2lo']=(('time3'),areas_list_merra2lo)
dk['grids_list_merra2lo']=(('time3'),grids_list_merra2lo)
dk['areas_list_merra2cndl']=(('time3'),areas_list_merra2cndl)
dk['grids_list_merra2cndl']=(('time3'),grids_list_merra2cndl)
dk['areas_list_merra2ac']=(('time3'),areas_list_merra2ac)
dk['grids_list_merra2ac']=(('time3'),grids_list_merra2ac)

dk['areas_list_jra55gw']=(('time3'),areas_list_jra55gw)
dk['grids_list_jra55gw']=(('time3'),grids_list_jra55gw)

dk.coords['time']=np.arange(1940,2024,1)
dk.coords['time1']=np.arange(1980,2024,1)
dk.coords['time2']=np.arange(1958,2024,1)
dk.coords['time3']=np.arange(1980,2020,1)

try:
    os.remove(dir_data+'land_area_intercomparison.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'land_area_intercomparison.nc',mode='w',format='NETCDF4')

In [ ]:
yrs=np.arange(1980,2020,1)
areas_list=[]

ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
t2m=ds['t2m'][0,:,:]
land_mask=t2m/t2m
land_mask=land_mask.fillna(0)
        
lon_east=[x for x in land_mask.longitude.values if x>=180.]
lon_west=[x for x in land_mask.longitude.values if x<180.]
l_east=land_mask.sel(longitude=lon_east)
l_west=land_mask.sel(longitude=lon_west)
l_east['longitude']=[x-360. for x in lon_east]
land_mask=xr.concat([l_east,l_west],dim='longitude')
        
dk=xr.Dataset()
dk['land']=(('lat','lon'),land_mask.values)
dk.coords['lat']=land_mask.latitude.values
dk.coords['lon']=land_mask.longitude.values
        
land_mask=dk['land']

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir2+'era_5_gridareas.nc')
    areas=ds['cell_area']/1e6
    ds=xr.open_dataset('/Users/ahenny/Downloads/ERA5.ar_tag.Reid500.1hr.'+str(year)+'0101-'+str(year)+'1231.nc')
    #ds=xr.open_dataset('/Users/ahenny/Downloads/ERA5.ar_tag.Mundhenk_v3.1hr.'+str(year)+'0101-'+str(year)+'1231.nc')
    #ds=xr.open_dataset('/Users/ahenny/Downloads/ERA5.ar_tag.TempestLR.1hr.'+str(year)+'.nc')
    #ds['lat']=ds1.lat.values
    #ds['lon']=ds1.lon.values
    ar=ds['ar_binary_tag'][0::6,:,:]
    
    #lons_west=[x for x in ar.lon.values if x<180]
    #lons_east=[x for x in ar.lon.values if x>=180]
    #ar_west=ar.sel(lon=lons_west)
    #ar_east=ar.sel(lon=lons_east)
    #ar_east['lon']=[x-360. for x in lons_east]
    #ar=xr.concat([ar_east,ar_west],dim='lon')
    
    
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    ar=ar.where(land_mask_interp>=0.5)
    
    area1=areas.where(ar==1).sum().values.tolist()
    areas_list.append(area1)
    
dk=xr.Dataset()
dk['area']=(('time'),areas_list)
dk.coords['time']=yrs
dk['area'].attrs["units"]='km2*yr-1'
dk.to_netcdf(dir_data+'era5reid_area_redo_land.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir_data+'land_area_intercomparison.nc')
grids_list_era5_variable=ds['grids_list_era5_variable'].values.tolist()
grids_list_era5_fixed=ds['grids_list_era5_fixed'].values.tolist()
grids_list_era5_fixed_poleward=ds['grids_list_era5_fixed_poleward'].values.tolist()

grids_list_merra2_variable=ds['grids_list_merra2_variable'].values.tolist()
grids_list_merra2_fixed=ds['grids_list_merra2_fixed'].values.tolist()
grids_list_merra2_fixed_poleward=ds['grids_list_merra2_fixed_poleward'].values.tolist()

grids_list_jra55_variable=ds['grids_list_jra55_variable'].values.tolist()
grids_list_jra55_fixed=ds['grids_list_jra55_fixed'].values.tolist()
grids_list_jra55_fixed_poleward=ds['grids_list_jra55_fixed_poleward'].values.tolist()

grids_list_era5tp=ds['grids_list_era5tp'].values.tolist()
grids_list_era5reid=ds['grids_list_era5reid'].values.tolist()
grids_list_era5mk=ds['grids_list_era5mk'].values.tolist()

grids_list_merra2lo=ds['grids_list_merra2lo'].values.tolist()
grids_list_merra2cndl=ds['grids_list_merra2cndl'].values.tolist()
grids_list_merra2ac=ds['grids_list_merra2ac'].values.tolist()

grids_list_jra55gw=ds['grids_list_jra55gw'].values.tolist()


areas_list_era5_variable=ds['areas_list_era5_variable'].values.tolist()
areas_list_era5_fixed=ds['areas_list_era5_fixed'].values.tolist()
areas_list_era5_fixed_poleward=ds['areas_list_era5_fixed_poleward'].values.tolist()

areas_list_merra2_variable=ds['areas_list_merra2_variable'].values.tolist()
areas_list_merra2_fixed=ds['areas_list_merra2_fixed'].values.tolist()
areas_list_merra2_fixed_poleward=ds['areas_list_merra2_fixed_poleward'].values.tolist()

areas_list_jra55_variable=ds['areas_list_jra55_variable'].values.tolist()
areas_list_jra55_fixed=ds['areas_list_jra55_fixed'].values.tolist()
areas_list_jra55_fixed_poleward=ds['areas_list_jra55_fixed_poleward'].values.tolist()

#areas_list_era5tp=ds['areas_list_era5tp'].values.tolist()
#areas_list_era5reid=ds['areas_list_era5reid'].values.tolist()
#areas_list_era5mk=ds['areas_list_era5mk'].values.tolist()

areas_list_merra2lo=ds['areas_list_merra2lo'].values.tolist()
areas_list_merra2cndl=ds['areas_list_merra2cndl'].values.tolist()
areas_list_merra2ac=ds['areas_list_merra2ac'].values.tolist()

areas_list_jra55gw=ds['areas_list_jra55gw'].values.tolist()

ds=xr.open_dataset(dir_data+'era5reid_area_redo_land.nc')
areas_list_era5reid=ds['area'].values.tolist()

ds=xr.open_dataset(dir_data+'era5tp_area_redo_land.nc')
areas_list_era5tp=ds['area'].values.tolist()

ds=xr.open_dataset(dir_data+'era5mk_area_redo_land.nc')
areas_list_era5mk=ds['area'].values.tolist()

In [ ]:
sigs_list=[]

yrs=np.arange(1940,2024,1)
lengths_list=[365.]*84
for i in range(len(yrs)):
    year=yrs[i]
    if year%4==0:
        lengths_list[i]=366.

new_era5_variable=[x/y*365. for x,y in zip (areas_list_era5_variable,lengths_list)]
new_era5_fixed=[x/y*365. for x,y in zip (areas_list_era5_fixed,lengths_list)]
new_era5_fixed_poleward=[x/y*365. for x,y in zip (areas_list_era5_fixed_poleward,lengths_list)]

yrs=np.arange(1980,2024,1)
lengths_list=[365.]*44
for i in range(len(yrs)):
    year=yrs[i]
    if year%4==0:
        lengths_list[i]=366.

new_merra2_variable=[x/y*365. for x,y in zip (areas_list_merra2_variable,lengths_list)]
new_merra2_fixed=[x/y*365. for x,y in zip (areas_list_merra2_fixed,lengths_list)]
new_merra2_fixed_poleward=[x/y*365. for x,y in zip (areas_list_merra2_fixed_poleward,lengths_list)]

yrs=np.arange(1958,2024,1)
lengths_list=[365.]*66
for i in range(len(yrs)):
    year=yrs[i]
    if year%4==0:
        lengths_list[i]=366.

new_jra55_variable=[x/y*365. for x,y in zip (areas_list_jra55_variable,lengths_list)]
new_jra55_fixed=[x/y*365. for x,y in zip (areas_list_jra55_fixed,lengths_list)]
new_jra55_fixed_poleward=[x/y*365. for x,y in zip (areas_list_jra55_fixed_poleward,lengths_list)]

list_all=new_era5_variable[40:]+new_era5_fixed[40:]+new_era5_fixed_poleward[40:]+new_jra55_variable[22:]+new_jra55_fixed[22:]+new_jra55_fixed_poleward[22:]+new_merra2_variable+new_merra2_fixed+new_merra2_fixed_poleward
print(len(list_all))
mean_all=float(sum(list_all))/float(len(list_all))
mean_all=mean_all/148326000./4./365.

test_array=np.zeros((9,84))
testa=[np.nan]*40+new_merra2_fixed
testb=[np.nan]*40+new_merra2_fixed_poleward
testc=[np.nan]*40+new_merra2_variable

testa1=[np.nan]*18+new_jra55_fixed
testb1=[np.nan]*18+new_jra55_fixed_poleward
testc1=[np.nan]*18+new_jra55_variable

testa2=new_era5_fixed
testb2=new_era5_fixed_poleward
testc2=new_era5_variable

test_array[0,:]=testa
test_array[1,:]=testb
test_array[2,:]=testc
test_array[3,:]=testa1
test_array[4,:]=testb1
test_array[5,:]=testc1
test_array[6,:]=testa2
test_array[7,:]=testb2
test_array[8,:]=testc2

avg_value=np.nanmean(test_array,axis=0)
min_value=np.nanmin(test_array,axis=0)
max_value=np.nanmax(test_array,axis=0)

yrs=np.arange(1980,2020,1)
lengths_list=[365.]*44
for i in range(len(yrs)):
    year=yrs[i]
    if year%4==0:
        lengths_list[i]=366.

new_era5tp=[x/y*365. for x,y in zip (areas_list_era5tp,lengths_list)]
new_era5reid=[x/y*365. for x,y in zip (areas_list_era5reid,lengths_list)]
new_era5mk=[x/y*365. for x,y in zip (areas_list_era5mk,lengths_list)]

new_merra2lo=[x/y*365. for x,y in zip (areas_list_merra2lo,lengths_list)]
new_merra2cndl=[x/y*365. for x,y in zip (areas_list_merra2cndl,lengths_list)]
new_merra2ac=[x/y*365. for x,y in zip (areas_list_merra2ac,lengths_list)]

new_jra55gw=[x/y*365. for x,y in zip (areas_list_jra55gw,lengths_list)]
    
yrs=np.arange(1980,2024,1)
count=0
test=new_era5_variable[40:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_era5_fixed[40:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_era5_fixed_poleward[40:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1
    

test=new_jra55_variable[22:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_jra55_fixed[22:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_jra55_fixed_poleward[22:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1


test=new_merra2_variable
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_merra2_fixed
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_merra2_fixed_poleward
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=avg_value
r=st.linregress(yrs,avg_value[40:])
change=r[0]*len(yrs)/mean_all*100./148326000./4./365.

yrs=np.arange(1980,2020,1)
r=st.linregress(yrs,new_era5tp)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_era5tp))/float(len(areas_list_era5tp))
change1=change1/mean*100.
mean=mean/148326000./4./365.25
label_1='TempestLR ERA5 (Mean='+str(round(mean,4))+' LSA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_era5reid)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_era5reid))/float(len(areas_list_era5reid))
change1=change1/mean*100.
mean=mean/148326000./4./365.25
label_2='Reid500 ERA5 (Mean='+str(round(mean,4))+' LSA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_era5mk)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_era5mk))/float(len(areas_list_era5mk))
change1=change1/mean*100.
mean=mean/148326000./4./365.25
label_3='Mundhenk_v3 ERA5 (Mean='+str(round(mean,4))+' LSA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_merra2lo)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_merra2lo))/float(len(areas_list_merra2lo))
change1=change1/mean*100.
mean=mean/148326000./4./365.25
label_4='Lora_v2 MERRA-2 (Mean='+str(round(mean,4))+' LSA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_merra2cndl)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_merra2cndl))/float(len(areas_list_merra2cndl))
change1=change1/mean*100.
mean=mean/148326000./4./365.25
label_5='ClimateNet_DL MERRA-2 (Mean='+str(round(mean,4))+' LSA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_merra2ac)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_merra2ac))/float(len(areas_list_merra2ac))
change1=change1/mean*100.
mean=mean/148326000./4./365.25
label_6='ARCONNECT_v2 MERRA-2 (Mean='+str(round(mean,4))+' LSA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_jra55gw)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_jra55gw))/float(len(areas_list_jra55gw))
change1=change1/mean*100.
mean=mean/148326000./4./365.25
label_7='GuanWaliser JRA-55 (Mean='+str(round(mean,4))+' LSA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'


label='Variants (Mean='+str(round(mean_all,4))+' LSA, '+'$\Delta$='+str(round(change,1))+'%, '+str(count)+' significant)'
    
#label='AR land area, '+str(count)+' significant'
yrs=np.arange(1940,2024,1)
#fig=plt.figure(figsize=(11,14))
fig=plt.figure(figsize=(8,16))
ax=plt.subplot(1,1,1)

ax.fill_between([1939,1979.5],0,8,color='gray',alpha=0.4)
width=1.5
ax.plot(np.arange(1940,2024,1),[x/148326000./4./365.*100. for x in new_era5_variable],color='k',linewidth=width,linestyle=':')
ax.plot(np.arange(1940,2024,1),[x/148326000./4./365.*100. for x in new_era5_fixed],color='k',linewidth=width,linestyle='--')
ax.plot(np.arange(1940,2024,1),[x/148326000./4./365.*100. for x in new_era5_fixed_poleward],color='k',linewidth=width)

ax.plot(np.arange(1980,2024,1),[x/148326000./4./365.*100. for x in new_merra2_variable],color='k',linewidth=width,linestyle=':')
ax.plot(np.arange(1980,2024,1),[x/148326000./4./365.*100. for x in new_merra2_fixed],color='k',linewidth=width,linestyle='--')
ax.plot(np.arange(1980,2024,1),[x/148326000./4./365.*100. for x in new_merra2_fixed_poleward],color='k',linewidth=width)

ax.plot(np.arange(1958,2024,1),[x/148326000./4./365.*100. for x in new_jra55_variable],color='k',linewidth=width,linestyle=':')
ax.plot(np.arange(1958,2024,1),[x/148326000./4./365.*100. for x in new_jra55_fixed],color='k',linewidth=width,linestyle='--')
ax.plot(np.arange(1958,2024,1),[x/148326000./4./365.*100. for x in new_jra55_fixed_poleward],color='k',linewidth=width)


ax.fill_between(yrs,min_value/148326000./4./365.*100.,max_value/148326000./4./365.*100.,color='k',alpha=0.3)

#plt.axhline(y=1,color='k',linewidth=1)
ax.plot(np.arange(1940,2024,1),[x/148326000./4./365.*100. for x in avg_value],color='k',linewidth=8,label=label,zorder=100)

yrs=np.arange(1980,2020,1)
width=6

ax.plot(yrs,[x/148326000./4./365.*100. for x in new_merra2lo],color='b',linewidth=width,label=label_4)
ax.plot(yrs,[x/148326000./4./365.*100. for x in new_merra2cndl],color='cyan',linewidth=width,label=label_5)
ax.plot(yrs,[x/148326000./4./365.*100. for x in new_merra2ac],color='mediumorchid',linewidth=width,label=label_6)

ax.plot(yrs,[x/148326000./4./365.*100. for x in new_era5tp],color='limegreen',linewidth=width,label=label_1)
ax.plot(yrs,[x/148326000./4./365.*100. for x in new_era5reid],color='saddlebrown',linewidth=width,label=label_2)
ax.plot(yrs,[x/148326000./4./365.*100. for x in new_era5mk],color='orange',linewidth=width,label=label_3)

ax.plot(yrs,[x/148326000./4./365.*100. for x in new_jra55gw],color='deeppink',linewidth=width,label=label_7)



ax.set_ylim(0,8)
ax.set_xlim(1939,2024)
ax.tick_params(labelsize=17)
ax.set_xlabel('Year',fontsize=20)
ax.set_ylabel('% land area',fontsize=20)
ax.grid(True,color='gray')
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{land}$'+' '+r'$\bf{area}$',fontsize=32,pad=5)
#plt.legend(loc='upper center',fontsize=15,ncol=1)
plt.legend(loc='upper center',fontsize=13,ncol=1)
plt.tight_layout()
plt.yticks([1,2,3,4,5,6,7])
#plt.yticks([0.7,0.8,0.9,1,1.1,1.2,1.3,1.4],labels=['-30%','-20%','-10%','0','+10%','+20%','+30%','+40%'])
for axis in ['bottom','top','left','right']:
    ax.spines[axis].set_linewidth(2)

plt.show()

In [ ]:
fig.savefig(dir_data+'area_land_redo_1a.png',dpi=200)

In [ ]:
test=[x/148326000./4./365. for x in areas_list_era5_fixed]
fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)
ax.plot(np.arange(1940,2024,1),test)
plt.axhline(y=0.02)
plt.show()

r=st.linregress(np.arange(1940,2024,1),test)
print(r)

In [ ]:
ds=xr.open_dataset(dir_data+'time_series_simple_era5.nc')
areas_list_era5_fixed=ds['area_list_era5_fixed'].values.tolist()
areas_list_era5_fixed_poleward=ds['area_list_era5_poleward'].values.tolist()
areas_list_era5_variable=ds['area_list_era5_variable'].values.tolist()

ds=xr.open_dataset(dir_data+'time_series_simple_merra2.nc')
areas_list_merra2_fixed=ds['area_list_merra2_fixed'].values.tolist()
areas_list_merra2_fixed_poleward=ds['area_list_merra2_poleward'].values.tolist()
areas_list_merra2_variable=ds['area_list_merra2_variable'].values.tolist()

ds=xr.open_dataset(dir_data+'time_series_simple_jra55.nc')
areas_list_jra55_fixed=ds['area_list_jra55_fixed'].values.tolist()
areas_list_jra55_fixed_poleward=ds['area_list_jra55_poleward'].values.tolist()
areas_list_jra55_variable=ds['area_list_jra55_variable'].values.tolist()


#ds=xr.open_dataset(dir_data+'time_series_simple_era5tp.nc')
#areas_list_era5tp=ds['area_list_era5tp'].values.tolist()

#ds=xr.open_dataset(dir_data+'time_series_simple_era5mk_fixed.nc')
#areas_list_era5mk=ds['area_list_era5mk'].values.tolist()

ds=xr.open_dataset(dir_data+'time_series_simple_merra2ac.nc')
areas_list_merra2ac=ds['area_list_merra2ac'].values.tolist()

ds=xr.open_dataset(dir_data+'time_series_simple_merra2lo.nc')
areas_list_merra2lo=ds['area_list_merra2lo'].values.tolist()

ds=xr.open_dataset(dir_data+'time_series_simple_jra55gw.nc')
areas_list_jra55gw=ds['area_list_jra55gw'].values.tolist()

ds=xr.open_dataset(dir_data+'time_series_simple_merra2cndl.nc')
areas_list_merra2cndl=ds['area_list_merra2cndl'].values.tolist()

#ds=xr.open_dataset(dir_data+'time_series_simple_era5reid.nc')
#areas_list_era5reid=ds['area_list_era5reid'].values.tolist()


ds=xr.open_dataset(dir_data+'era5reid_area_redo.nc')
areas_list_era5reid=ds['area'].values.tolist()

ds=xr.open_dataset(dir_data+'era5tp_area_redo.nc')
areas_list_era5tp=ds['area'].values.tolist()

ds=xr.open_dataset(dir_data+'era5mk_area_redo.nc')
areas_list_era5mk=ds['area'].values.tolist()

In [ ]:
sigs_list=[]

yrs=np.arange(1940,2024,1)
lengths_list=[365.]*84
for i in range(len(yrs)):
    year=yrs[i]
    if year%4==0:
        lengths_list[i]=366.

new_era5_variable=[x/y*365. for x,y in zip (areas_list_era5_variable,lengths_list)]
new_era5_fixed=[x/y*365. for x,y in zip (areas_list_era5_fixed,lengths_list)]
new_era5_fixed_poleward=[x/y*365. for x,y in zip (areas_list_era5_fixed_poleward,lengths_list)]

yrs=np.arange(1980,2024,1)
lengths_list=[365.]*44
for i in range(len(yrs)):
    year=yrs[i]
    if year%4==0:
        lengths_list[i]=366.

new_merra2_variable=[x/y*365. for x,y in zip (areas_list_merra2_variable,lengths_list)]
new_merra2_fixed=[x/y*365. for x,y in zip (areas_list_merra2_fixed,lengths_list)]
new_merra2_fixed_poleward=[x/y*365. for x,y in zip (areas_list_merra2_fixed_poleward,lengths_list)]

yrs=np.arange(1958,2024,1)
lengths_list=[365.]*66
for i in range(len(yrs)):
    year=yrs[i]
    if year%4==0:
        lengths_list[i]=366.

new_jra55_variable=[x/y*365. for x,y in zip (areas_list_jra55_variable,lengths_list)]
new_jra55_fixed=[x/y*365. for x,y in zip (areas_list_jra55_fixed,lengths_list)]
new_jra55_fixed_poleward=[x/y*365. for x,y in zip (areas_list_jra55_fixed_poleward,lengths_list)]

list_all=new_era5_variable[40:]+new_era5_fixed[40:]+new_era5_fixed_poleward[40:]+new_jra55_variable[22:]+new_jra55_fixed[22:]+new_jra55_fixed_poleward[22:]+new_merra2_variable+new_merra2_fixed+new_merra2_fixed_poleward
print(len(list_all))
mean_all=float(sum(list_all))/float(len(list_all))
mean_all=mean_all/509600000./4./365.

test_array=np.zeros((9,84))
testa=[np.nan]*40+new_merra2_fixed
testb=[np.nan]*40+new_merra2_fixed_poleward
testc=[np.nan]*40+new_merra2_variable

testa1=[np.nan]*18+new_jra55_fixed
testb1=[np.nan]*18+new_jra55_fixed_poleward
testc1=[np.nan]*18+new_jra55_variable

testa2=new_era5_fixed
testb2=new_era5_fixed_poleward
testc2=new_era5_variable

test_array[0,:]=testa
test_array[1,:]=testb
test_array[2,:]=testc
test_array[3,:]=testa1
test_array[4,:]=testb1
test_array[5,:]=testc1
test_array[6,:]=testa2
test_array[7,:]=testb2
test_array[8,:]=testc2

avg_value=np.nanmean(test_array,axis=0)
min_value=np.nanmin(test_array,axis=0)
max_value=np.nanmax(test_array,axis=0)


yrs=np.arange(1980,2020,1)
lengths_list=[365.]*44
for i in range(len(yrs)):
    year=yrs[i]
    if year%4==0:
        lengths_list[i]=366.

new_era5tp=[x/y*365. for x,y in zip (areas_list_era5tp,lengths_list)]
new_era5reid=[x/y*365. for x,y in zip (areas_list_era5reid,lengths_list)]
new_era5mk=[x/y*365. for x,y in zip (areas_list_era5mk,lengths_list)]

new_merra2lo=[x/y*365. for x,y in zip (areas_list_merra2lo,lengths_list)]
new_merra2cndl=[x/y*365. for x,y in zip (areas_list_merra2cndl,lengths_list)]
new_merra2ac=[x/y*365. for x,y in zip (areas_list_merra2ac,lengths_list)]

new_jra55gw=[x/y*365. for x,y in zip (areas_list_jra55gw,lengths_list)]
    
    
yrs=np.arange(1980,2024,1)
count=0
test=new_era5_variable[40:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_era5_fixed[40:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_era5_fixed_poleward[40:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1
    

test=new_jra55_variable[22:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_jra55_fixed[22:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_jra55_fixed_poleward[22:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1


test=new_merra2_variable
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_merra2_fixed
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_merra2_fixed_poleward
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=avg_value
r=st.linregress(yrs,avg_value[40:])
change=r[0]*len(yrs)/mean_all*100./509600000./4./365.25

yrs=np.arange(1980,2020,1)
r=st.linregress(yrs,new_era5tp)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_era5tp))/float(len(areas_list_era5tp))
change1=change1/mean*100.
mean=mean/509600000./4./365.25
label_1='TempestLR ERA5 (Mean='+str(round(mean,4))+' ESA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_era5reid)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_era5reid))/float(len(areas_list_era5reid))
change1=change1/mean*100.
mean=mean/509600000./4./365.25
label_2='Reid500 ERA5 (Mean='+str(round(mean,4))+' ESA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_era5mk)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_era5mk))/float(len(areas_list_era5mk))
change1=change1/mean*100.
mean=mean/509600000./4./365.25
label_3='Mundhenk_v3 ERA5 (Mean='+str(round(mean,4))+' ESA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_merra2lo)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_merra2lo))/float(len(areas_list_merra2lo))
change1=change1/mean*100.
mean=mean/509600000./4./365.25
label_4='Lora_v2 MERRA-2 (Mean='+str(round(mean,4))+' ESA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_merra2cndl)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_merra2cndl))/float(len(areas_list_merra2cndl))
change1=change1/mean*100.
mean=mean/509600000./4./365.25
label_5='ClimateNet_DL MERRA-2 (Mean='+str(round(mean,4))+' ESA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_merra2ac)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_merra2ac))/float(len(areas_list_merra2ac))
change1=change1/mean*100.
mean=mean/509600000./4./365.25
label_6='ARCONNECT_v2 MERRA-2 (Mean='+str(round(mean,4))+' ESA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_jra55gw)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(areas_list_jra55gw))/float(len(areas_list_jra55gw))
change1=change1/mean*100.
mean=mean/509600000./4./365.25
label_7='GuanWaliser JRA-55 (Mean='+str(round(mean,4))+' ESA, '+'$\Delta$='+str(round(change1,1))+'%, p='+str(round(r[3],3))+')'


label='Variants (Mean='+str(round(mean_all,4))+' ESA, '+'$\Delta$='+str(round(change,1))+'%, '+str(count)+' significant)'
    
#label='AR land area, '+str(count)+' significant'
yrs=np.arange(1940,2024,1)
#fig=plt.figure(figsize=(11,14))
fig=plt.figure(figsize=(8,16))
ax=plt.subplot(1,1,1)

ax.fill_between([1939,1979.5],0,13,color='gray',alpha=0.4)
width=1.5
ax.plot(np.arange(1940,2024,1),[x/509600000./4./365.*100. for x in new_era5_variable],color='k',linewidth=width,linestyle=':')
ax.plot(np.arange(1940,2024,1),[x/509600000./4./365.*100. for x in new_era5_fixed],color='k',linewidth=width,linestyle='--')
ax.plot(np.arange(1940,2024,1),[x/509600000./4./365.*100. for x in new_era5_fixed_poleward],color='k',linewidth=width)

ax.plot(np.arange(1980,2024,1),[x/509600000./4./365.*100. for x in new_merra2_variable],color='k',linewidth=width,linestyle=':')
ax.plot(np.arange(1980,2024,1),[x/509600000./4./365.*100. for x in new_merra2_fixed],color='k',linewidth=width,linestyle='--')
ax.plot(np.arange(1980,2024,1),[x/509600000./4./365.*100. for x in new_merra2_fixed_poleward],color='k',linewidth=width)

ax.plot(np.arange(1958,2024,1),[x/509600000./4./365.*100. for x in new_jra55_variable],color='k',linewidth=width,linestyle=':')
ax.plot(np.arange(1958,2024,1),[x/509600000./4./365.*100. for x in new_jra55_fixed],color='k',linewidth=width,linestyle='--')
ax.plot(np.arange(1958,2024,1),[x/509600000./4./365.*100. for x in new_jra55_fixed_poleward],color='k',linewidth=width)


ax.fill_between(yrs,min_value/509600000./4./365.*100.,max_value/509600000./4./365.*100.,color='k',alpha=0.3)

#plt.axhline(y=1,color='k',linewidth=1)
ax.plot(np.arange(1940,2024,1),[x/509600000./4./365.*100. for x in avg_value],color='k',linewidth=8,label=label,zorder=100)

yrs=np.arange(1980,2020,1)
width=6

ax.plot(yrs,[x/509600000./4./365.*100. for x in new_merra2lo],color='b',linewidth=width,label=label_4)
ax.plot(yrs,[x/509600000./4./365.*100. for x in new_merra2cndl],color='cyan',linewidth=width,label=label_5)
ax.plot(yrs,[x/509600000./4./365.*100. for x in new_merra2ac],color='mediumorchid',linewidth=width,label=label_6)

ax.plot(yrs,[x/509600000./4./365.*100. for x in new_era5tp],color='limegreen',linewidth=width,label=label_1)
ax.plot(yrs,[x/509600000./4./365.*100. for x in new_era5reid],color='saddlebrown',linewidth=width,label=label_2)
ax.plot(yrs,[x/509600000./4./365.*100. for x in new_era5mk],color='orange',linewidth=width,label=label_3)

ax.plot(yrs,[x/509600000./4./365.*100. for x in new_jra55gw],color='deeppink',linewidth=width,label=label_7)



ax.set_ylim(0,13)
ax.set_xlim(1939,2024)
ax.tick_params(labelsize=17)
ax.set_xlabel('Year',fontsize=20)
ax.set_ylabel('% Earth surface area',fontsize=20)
ax.grid(True,color='gray')
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{global}$'+' '+r'$\bf{area}$',fontsize=32,pad=5)
#plt.legend(loc='upper center',fontsize=15,ncol=1)
plt.legend(loc='upper center',fontsize=13,ncol=1)
plt.tight_layout()
plt.yticks([2,4,6,8,10,12])
#plt.yticks([0.7,0.8,0.9,1,1.1,1.2,1.3,1.4],labels=['-30%','-20%','-10%','0','+10%','+20%','+30%','+40%'])
for axis in ['bottom','top','left','right']:
    ax.spines[axis].set_linewidth(2)

plt.show()

In [ ]:
fig.savefig(dir_data+'area_land_redo_2a.png',dpi=200)

In [ ]:
print(sigs_list)